# 1. 데이터 불러오기

In [14]:
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [36]:
df_total = pd.read_csv('df_total.csv', index_col=0)
df_test = pd.read_csv('df_test.csv', index_col=0)

# 2. 데이터 전처리

In [37]:
# 기준일 ID -> 날짜 데이터 추출
df_total['기준일ID'] = pd.to_datetime(df_total['기준일ID'])
df_test['기준일ID'] = pd.to_datetime(df_test['기준일ID'])

In [38]:
df_total['year'] = df_total['기준일ID'].dt.year
df_total['month'] = df_total['기준일ID'].dt.month
df_total['day'] = df_total['기준일ID'].dt.day

df_test['year'] = df_test['기준일ID'].dt.year
df_test['month'] = df_test['기준일ID'].dt.month
df_test['day'] = df_test['기준일ID'].dt.day

In [39]:
# 불필요한 열 제거 및 열 이름 변경('총생활인구수' 외에 불필요한 열 제거)
# 시간 데이터 남기고, '시간대구분' column 을 'hour'로 rename

df_total_process = df_total[['year', 'month', 'day', '휴일', '시간대구분', '총생활인구수']]
df_test_process = df_test[['year', 'month', 'day', '휴일', '시간대구분', '총생활인구수']]

df_total_process.rename(columns={'시간대구분': 'hour'})
df_test_process.rename(columns={'시간대구분': 'hour'})
df_total_process.head(1)

,year,month,day,휴일,시간대구분,총생활인구수
0,2017,1,1,1,0,31535.22


In [40]:
# rolling 및 shift -> 일주일 이동 평균을 계산하기 위해 rolling_7 변수 추가
df_total_process['rolling_7'] = df_total_process['총생활인구수'].rolling(window=7*24, min_periods=7*24).mean()
df_test_process['rolling_7'] = df_total_process['총생활인구수'].rolling(window=7*24, min_periods=7*24).mean()

# 일주일 전 data와 비교하기 위해 shift 수행
df_total_process['shift_7'] = df_total_process['총생활인구수'].shift(7*24)
df_test_process['shift_7'] = df_test_process['총생활인구수'].shift(7*24)

# 결측치 제거 -> 행 단위로 제거했음(영향이 크게 없다 생각하였음)
df_total_process.dropna(axis=0, inplace=True)
df_test_process.dropna(axis=0, inplace=True)

df_total_process.reset_index(drop=True, inplace=True)
df_test_process.reset_index(drop=True, inplace=True)

In [42]:
# train, test 데이터를 train_x, train_y, test_x, test_y로 나누기
target = '총생활인구수'

train_x = df_total_process.drop(target, axis=1)
train_y = df_total_process.loc[:, target]
test_x = df_test_process.drop(target, axis=1)
test_y = df_test_process.loc[:, target]

In [46]:
# 데이터 저장
train_x.to_csv('train_x.csv', index = False)
train_y.to_csv('train_y.csv', index = False)
test_x.to_csv('test_x.csv', index = False)
test_y.to_csv('test_y.csv', index = False)